In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv('strokeData.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [23]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [24]:
mean1 = df['bmi'].mean()
print(mean1)
df = df.fillna({'bmi': mean1})
#df = df.interpolate()

28.893236911794666


In [25]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.698018,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.800000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.400000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,32.800000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


check for outliers here and fix them

In [26]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df['ever_married']= label_encoder.fit_transform(df['ever_married'])
df['work_type']= label_encoder.fit_transform(df['work_type'])
df['Residence_type']=label_encoder.fit_transform(df['Residence_type'])
df['smoking_status']=label_encoder.fit_transform(df['smoking_status'])
df['gender']=label_encoder.fit_transform(df['gender'])

df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


In [27]:
df = df.drop_duplicates()

In [28]:
df.stroke.value_counts()

stroke
0    4861
1     249
Name: count, dtype: int64

Data is imbalanced

fix 1 oversampling  (just have to use df_test_over instead of df)

In [29]:
count_class_0, count_class_1 = df.stroke.value_counts()

df_class_0 = df[df['stroke'] == 0]
df_class_1 = df[df['stroke'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.stroke.value_counts())

Random over-sampling:
stroke
0    4861
1    4861
Name: count, dtype: int64


In [30]:
df = df.drop('id', axis='columns')

In [31]:
Y = df["stroke"]
X = df.drop('stroke', axis='columns')

Below is the fix 2 of oversampling SMOTE (better than oversampling) use X_sm and y_sm instead

In [32]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, Y)

y_sm.value_counts()

stroke
1    4861
0    4861
Name: count, dtype: int64

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sm)
X_scaled

array([[ 1.43363584,  0.53791694, -0.30572095, ...,  2.01301753,
         1.09412937, -0.28773269],
       [-0.69687587,  0.26707584, -0.30572095, ...,  1.53176084,
        -0.07618297,  0.74985529],
       [ 1.43363584,  1.12473934, -0.30572095, ..., -0.21824698,
         0.4715229 ,  0.74985529],
       ...,
       [ 1.43363584,  0.23521642, -0.30572095, ...,  1.3642342 ,
         0.45383813, -1.32532066],
       [-0.69687587,  1.03445897, -0.30572095, ..., -0.18772304,
         0.44226906, -0.28773269],
       [-0.69687587, -0.29845557, -0.30572095, ..., -0.800359  ,
        -0.99977939, -0.28773269]])

In [34]:
from sklearn.model_selection import train_test_split,GridSearchCV

x_train, x_test, y_train, y_test = train_test_split(X_scaled, y_sm, test_size=0.30, shuffle=True, random_state=10, stratify=y_sm)

here we used stratify=Y because there is imbalance so that xtrain and xtest both are balanced when samples divided among them

NOTE X and Y have been interchanged from AKpython

In [35]:
y_train.value_counts()

stroke
0    3403
1    3402
Name: count, dtype: int64

MODELS EVALUATION

In [36]:
from sklearn.model_selection import cross_val_score

Decision Tree Classifier    (Note can also use bagged DecisionTreeClassifier)

In [37]:
from sklearn.tree import DecisionTreeClassifier

clf1 = DecisionTreeClassifier(max_depth=5)
clf1.fit(x_train, y_train)
score_normal_1 = clf1.score(x_test, y_test)
print(score_normal_1)

scores_cross_1 = cross_val_score(DecisionTreeClassifier(), X_scaled, y_sm, cv=5)
print(scores_cross_1.mean())

0.8165923894412067
0.9034173304981646


SVM  (do it for poly and rf as well using grid cross cv and also try bagging them)

In [38]:
from sklearn.svm import SVC

clf2 = SVC(kernel="linear", C=0.025)
clf2.fit(x_train, y_train)
score_normal_2 = clf2.score(x_test, y_test)
print(score_normal_2)

scores_cross_2 = cross_val_score(SVC(), X_scaled, y_sm, cv=5)
print(scores_cross_2.mean())

0.7943092218032225
0.8533252404075027


KNeighbors Classifier

In [39]:
from sklearn.neighbors import KNeighborsClassifier

clf3 = KNeighborsClassifier(3)
clf3.fit(x_train, y_train)
score_normal_3 = clf3.score(x_test, y_test)
print(score_normal_3)

scores_cross_3 = cross_val_score(KNeighborsClassifier(), X_scaled, y_sm, cv=5)
print(scores_cross_3.mean())

0.8902982516283853
0.8763660647222486


Naive Bayes Classifier

In [40]:
from sklearn.naive_bayes import GaussianNB

clf4 = GaussianNB()
clf4.fit(x_train, y_train)
score_normal_4 = clf4.score(x_test, y_test)
print(score_normal_4)

scores_cross_4 = cross_val_score(GaussianNB(), X_scaled, y_sm, cv=5)
print(scores_cross_4.mean())

0.7860815906753514
0.7849220857532768


MLPClassifier

In [41]:
from sklearn.neural_network import MLPClassifier

clf5 = MLPClassifier(alpha=1, max_iter=1000)
clf5.fit(x_train, y_train)
score_normal_5 = clf5.score(x_test, y_test)
print(score_normal_5)

scores_cross_5 = cross_val_score(MLPClassifier(alpha=1, max_iter=1000), X_scaled, y_sm, cv=5)
print(scores_cross_5.mean())

0.8248200205690778
0.8355307478286627


SGDClassifier

In [42]:
from sklearn.linear_model import SGDClassifier

clf6 = SGDClassifier(loss="hinge", penalty="l2")
clf6.fit(x_train, y_train)
score_normal_6 = clf6.score(x_test, y_test)
print(score_normal_6)

scores_cross_6 = cross_val_score(SGDClassifier(), X_scaled, y_sm, cv=5)
print(scores_cross_6.mean())

0.7836818649297224
0.7875961894485174


Logistic Regression

In [43]:
from sklearn.linear_model import LogisticRegression

clf7 = LogisticRegression(max_iter = 1000)
clf7.fit(x_train, y_train)
score_normal_7 = clf7.score(x_test, y_test)
print(score_normal_7)

scores_cross_7 = cross_val_score(SGDClassifier(), X_scaled, y_sm, cv=5)
print(scores_cross_7.mean())

0.7943092218032225
0.7931521152686535


In [52]:
from sklearn.ensemble import RandomForestClassifier
clf8 = RandomForestClassifier(max_depth=2, random_state=0)
clf8.fit(x_train, y_train)
y_pred=clf8.predict(x_test)

print(clf8.score(x_test,y_test))

scores_cross_8 = cross_val_score(RandomForestClassifier(), X_scaled, y_sm, cv=5)
print(scores_cross_8.mean())

0.777511141583819
0.9414755572481937
